In [6]:
import gograph
import similarity
import importlib
import re
import time
import annotations

import pandas as pd
import numpy as np

In [7]:
def reload_my_modules():
    """Re-import my modules into the IPython shell"""
    importlib.reload(annotations)
    importlib.reload(gograph)
    importlib.reload(similarity)

In [8]:
%%time
reload_my_modules()
fp = 'data/goa_human.gaf'
anno = annotations.Annotations(fp)

/usr/local/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


CPU times: user 1.28 s, sys: 180 ms, total: 1.46 s
Wall time: 1.48 s


In [9]:
# num proteins
print(anno.annotations.groupby('DB_Object_ID').size().count())
# num annos per protein
print(anno.annotations.groupby('DB_Object_ID').size().mean())
#
print(anno.annotations.groupby('DB_Object_ID').size().quantile([0, 0.25, 0.5, 0.75, 1]))
# total annos
print(len(anno.annotations))

19626
25.621216753286458
0.00      1.0
0.25      7.0
0.50     15.0
0.75     30.0
1.00    914.0
dtype: float64
502842


In [10]:
%%time
reload_my_modules()
fp = 'data/go-basic.obo'
#fp = 'data/gene_ontology_edit.obo.2019-07-01'
ontology = gograph.OBOParser(fp).parse_ontology()

CPU times: user 2.35 s, sys: 133 ms, total: 2.48 s
Wall time: 2.52 s


In [11]:
t = time.time()
ontology.make_ancestry_matrix()
print(time.time()-t)

10.222615718841553


In [12]:
ontology.ancestry_matrix.matrix.setdiag(1)

In [13]:
anno.annotations = anno.annotations.loc[anno.annotations.Aspect == 'P', :]

In [14]:
len(anno.annotations)

164407

In [15]:
importlib.reload(annotations)
ft = annotations.FrequencyTable(anno, ontology.ancestry_matrix)

/Users/ilyanovikov/Documents/CodeRepos/ggid/annotations.py:52: RuntimeWarning: divide by zero encountered in log10
  self.information_content = -1*np.log10(self.deep_count.sum(axis=0)/self.deep_count.sum())


In [16]:
p = pd.read_csv('data/list_of_human_kinases.csv')
p.head()

,Unnamed: 0,gene_symbol,uniprot
0,0,AKT1,P31749
1,1,AKT2,P31751
2,2,AKT3,Q9Y243
3,3,MRCKA,Q5VT25
4,4,MRCKB,Q9Y5S2


In [87]:
reload_my_modules()

#P25098 GRK2
#P35626 GRK3

sim = similarity.Similarity(anno, ontology, ft, list(p.gene_symbol), namespace='P', verbose=True)
sim.calculate_similarity()

333


In [88]:
sim.sim_matrix.threshold_matrix()
sim.sim_matrix.adj_matrix

<333x333 sparse matrix of type '<class 'numpy.float64'>'
	with 6327 stored elements in COOrdinate format>

In [89]:
sim.sim_matrix.save_as_pickle('results/kinase_matrix.pkl')

In [91]:
len(sim.sim_matrix.protein_list)

333

In [18]:
sim.calculate_similarity_mult_cpu()

worker started
worker started
worker done 114.53437519073486
worker done 114.82369208335876


In [21]:
print(sim.workers)

[<Process(Process-1, started)>, <Process(Process-2, started)>]


In [30]:
t0 = time.time()
print(time.time() - t0)
print(sim.counter)

116.69745802879333
62941443


In [33]:
type(sim.sim_matrix)#.argsort(axis=0) < 2

scipy.sparse.coo.coo_matrix

In [39]:
sim.sim_matrix = smt

In [40]:
sim.threshold_matrix()

AttributeError: 'Similarity' object has no attribute 'threshold_matrix'

In [105]:
#pickle.dump(smt3, open('results/kinase_matrix.pkl', 'wb'))

In [81]:
import pickle
abc = pickle.load(open('results/kinase_matrix.pkl', 'rb'))

In [83]:
abc.__dict__

{'sim_matrix': <5x5 sparse matrix of type '<class 'numpy.float64'>'
 	with 15 stored elements in COOrdinate format>,
 'protein_list': ['AKT1', 'AKT2', 'AKT3', 'DMPK', 'GRK1']}